# EDA
Como paso inicial se importa las librerias a utilizar en nuestro proceso de exploración de los datos que se encuentran comprimidos en formato .gz; estos datos vienen en un formato json, se va a convertirlos en data frames haciendo uso de pandas para ser guardados en un formato csv.

In [1]:
# *** HENRY ***
# *** PROYECTO INDIVIDUAL 01 ***
# *** Sistema de Recomendación de Videojuegos ***
# *** MLOps Engineer ***

# ARCHIVO 'user_reviews.json.gz'

# LIBRERÍAS
import pandas as pd
import gzip
import ast
import json

data = []
with gzip.open("C:\\Users\\PC\\Desktop\\HENRY__PI__01\\Base de datos\\user_reviews.json.gz", 'rb') as f:
  for line in f:
    data.append(ast.literal_eval(line.decode('utf-8')))
reviews = pd.DataFrame(data)
f.close()

In [2]:
# Desempaquetado de la columna 'reviews'

reviews_data = []
for index, row in reviews.iterrows():
    user_id = row['user_id']
    user_url = row['user_url']
    reviews_data.extend([{'user_id': user_id, 'user_url': user_url, **review_value} for review_value in row['reviews']])
reviews = pd.DataFrame(reviews_data)

In [4]:
# Mostrando las 5 primeras filas del dataframe
reviews.head()

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


In [5]:
# Obteniendo informacion acerca del DataFrame

reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      59305 non-null  object
 1   user_url     59305 non-null  object
 2   funny        59305 non-null  object
 3   posted       59305 non-null  object
 4   last_edited  59305 non-null  object
 5   item_id      59305 non-null  object
 6   helpful      59305 non-null  object
 7   recommend    59305 non-null  bool  
 8   review       59305 non-null  object
dtypes: bool(1), object(8)
memory usage: 3.7+ MB


##### Descripción del DataFrame:
Se puede observar que el DataFrame tiene 9 columnas y 59305 filas.
También se puede notar que no existen filas donde todas las columnas son NaN y que se pueden borrar.
Explorando las columnas nos podemos dar cuenta que para nuestro objetivo no es necesario la columna 'playtime_2weeks', por lo que se pueden borrar.

Por otro lado, se hace una pequeña descripción de cada una de las columnas:
- user_id: es un identificador único para cada usuario.
- user_url: es la URL del usuario en streamcommunity.
- funny: muestra si alguien colocó un emoticon al review.
- posted: es la fecha del posteo del review.
- last_edited: es la fecha de la ultima edicion.
- item_id: es el identificador único del juego.
- helpful: es la muestra estadística donde otros usuarios indican si fue útil la información.
- recommend: es un booleano que indica si el usuario recomienda o no el juego.
- review: es un string con los comentarios sobre el juego.

In [6]:
# Eliminando filas duplicadas

reviews = reviews.drop_duplicates()

In [7]:
# Reemplazo de datos vacíos por '0' y extrayendo sólo los números y transformandolo a entero en la columna 'funny'

reviews['funny'] = reviews['funny'].replace('', '0')
reviews['funny'] = reviews['funny'].str.extract('(\d+)').astype(int)

In [8]:
# Escribiendo 'sin editar' en las celdas sin datos en la columna 'last_edited'

reviews['last_edited'] = reviews['last_edited'].replace('', 'Sin Editar')

In [9]:
# Cambiando el tipo de dato de object a int en la columna 'item_id'

reviews['item_id'] = reviews['item_id'].astype(int)

In [10]:
# Creación de la columna 'sentiment_analysis'


import zipfile, gzip, json, ast, nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

reviews['sentiment_analysis'] = reviews['review'].apply(lambda x: SentimentIntensityAnalyzer().polarity_scores(x)['compound'])

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [11]:
# Estandarización a 1, 2 3

condiciones = [reviews['sentiment_analysis'] > 0.11,
              reviews['sentiment_analysis'] < -0.25]
valores = [2, 0]
reviews['sentiment_analysis'] = np.select(condiciones, valores, default=1)

In [13]:
# Elimina todas las filas en donde aparece al menos un NaN
reviews.dropna(inplace=True)

In [ ]:
# Guardando el archivo limpio en formato csv

reviews.to_csv('reviews_limpio.csv', index=False)
reviews.to_parquet('reviews_limpio.parquet')